In [10]:
import scipy
import scipy.io
import pickle
import numpy as np
from scipy.linalg import norm
import matplotlib.pyplot as plt
from sklearn.datasets import load_svmlight_file

In [21]:
W, y = load_svmlight_file('./data/cod-rna')

In [22]:
# параметры алгоритма
n = W.shape[0]
p = W.shape[1]
lam  = np.max(np.sqrt(np.sum(W.power(2), axis=1)))**2 / (4 * n)
x0 = np.zeros(p) # надо еще одну размерность?
bias = 0
terminate_tol = 1e-6
steps_tol = 0.25
n_steps = 1000
alpha = 2 # alpha = 3
R = 1
printst = 100

In [23]:
def projection_simplex_sort(v, z=1):
    n_features = v.shape[0]
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - z
    ind = np.arange(n_features) + 1
    cond = u - cssv / ind > 0
    rho = ind[cond][-1]
    theta = cssv[cond][-1] / float(rho)
    w = np.maximum(v - theta, 0)
    return w

def l1_ball_projection(y, z=R):
    if norm(y, 1) <= R:
        return y
    pos_proj = projection_simplex_sort(np.abs(y), z)
    return np.sign(y) * pos_proj

In [24]:
def fn_logis_Grad(y, W, bias, x, lam):
    Top = np.exp(-y * ( W * x  + bias)) # why minus?
    Rat = Top / (1 + Top)
    LM = -Rat * y
    ResM = scipy.sparse.diags(LM) * W
    Grad = ResM.sum(axis=0).A1 + lam * n * x
    return Grad, Top

In [25]:
def powereig2(svec, M):  
    # A = M' * M
    d = svec
    for i in range(1, 21):
        dv = M.transpose() * ( M * d)
        d = dv / norm(d)
    maxeig = dv.dot(d) / (d.dot(d))
    return maxeig

In [28]:
x_cur = x0
y_cur = x_cur
mu = lam * n

if p < n:
    M = W.copy()
    svec = np.ones(p)
else:
    M  = W.transpose()
    svec = np.ones(n)

maxeig = powereig2(svec, M)
L  = 0.25 * maxeig + mu

points = []
points.append(x_cur)

print('********* Algorithm starts *********')

for i in range(1, n_steps + 1):
    
    # Evaluate the gradient.
    grad_fx, _ = fn_logis_Grad(y, W, bias, y_cur, lam)
    if i == 1:
        norm_grad0 = max(1, norm(grad_fx.flatten(), 2))
    
    # Check the stopping criterion.
    rdiff = norm(grad_fx.flatten(), 2) / norm_grad0
    if (rdiff <= terminate_tol) and i > 1:
        print('Convergence achieved!')
        break
    
    
    x_nxt = l1_ball_projection(y_cur - (1 / L) * grad_fx)
    s = (np.sqrt(L) - np.sqrt(mu)) / (np.sqrt(L) + np.sqrt(mu))
    #s = (i-1) / (i +2)
    y_cur = x_nxt + s * (x_nxt - x_cur)
    x_cur = x_nxt
    
    points.append(x_cur)
    # Print the iteration.
    if i % printst  == 0:
        print('iter = %4d, rdiff = %3.3e\n' % (i, rdiff))
        
if i >= n_steps:
    print('Exceed the maximum number of iterations')

********* Algorithm starts *********
iter =  100, rdiff = 3.385e-02

iter =  200, rdiff = 5.456e-03

iter =  300, rdiff = 5.275e-04

iter =  400, rdiff = 4.583e-05

iter =  500, rdiff = 4.040e-05

iter =  600, rdiff = 1.360e-05

Convergence achieved!


In [29]:
with open('./AGD_points.pckl', 'wb') as f:
    pickle.dump(points, f)